In [6]:
import pandas as pd
import numpy as np
import datetime
import pickle
import json
import os

In [12]:
query_date = datetime.datetime.strptime('2018-12-08 00:00:00', '%Y-%m-%d %H:%M:%S')
province_list = os.listdir('data')
for province in province_list:
    city_list = os.listdir(os.path.join('data', province))
    if os.path.exists('result/%s.csv' % (province)): continue
    with open('result/%s.csv' % (province), 'w') as fp:
        fp.write('\t'.join(['from_province','from_city','from_area','to_province','to_city','to_area',
                           'jzkh_h_unit_fee','jzkh_l_unit_fee','jzkh_low_fee','jzkh_day','jzqy_h_unit_fee',
                           'jzqy_l_unit_fee','jzqy_low_fee','jzqy_day','th_below_fee','th_unit_fee','th_low_fee',
                           'th_day','tz_below_fee','tz_unit_fee','tz_low_fee','tz_day']))
        fp.write('\n')
    for city in city_list:
#         if city != u'朝阳市.pkl': continue
        print('Processing %s/%s...' % (province, city), end='')
        with open('data/%s/%s'%(province, city),'rb') as fp:
            data = pickle.load(fp)
        city_data_length = len(data)
        with open('result/%s.csv' % (province), 'a') as fp:
            for i in range(city_data_length): # 
                data_line = []
                prov, city, area = data[i]['fromAddr'].split('-')
#                 if area != u'北票市': continue
                #print(prov, city, area)
                data_line.append(prov) # from_province.append(prov)
                data_line.append(city) # from_city.append(city)
                data_line.append(area) # from_area.append(area)
                prov, city, area = data[i]['toAddr'].split('-')
#                 if area not in [u'弥渡县',u'天心区']: continue
                data_line.append(prov) # to_province.append(prov)
                data_line.append(city) # to_city.append(city)
                data_line.append(area) # to_area.append(area)
#                 print('->', prov, city, area)
                if isinstance(data[i].get('logisticsPublishPrices'), list):
                    target_counter = 0
                    for logit_price in data[i]['logisticsPublishPrices']:
                        if target_counter >= 2: break
#                         print(':\n')
                        if logit_price.__contains__('productName') and logit_price['productName'].find(u'精准卡航') >= 0:
                            if logit_price.get('generateRate') is not None:
                                if logit_price['generateRate'].get('receiveHeavyRate') is not None:
                                    data_line.append(logit_price['generateRate']['receiveHeavyRate']) # jzkh_h_unit_fee
#                                     print("1. jzkh_h_unit_fee=%d\n"%logit_price['generateRate']['receiveHeavyRate'])
                                else:
                                    data_line.append(logit_price['generateRate']['heavyRate']) # jzkh_h_unit_fee
#                                     print("2. jzkh_h_unit_fee=%d\n" % logit_price['generateRate']['heavyRate'])
                                # --------------------
                                if logit_price['generateRate'].get('receiveLightRate') is not None:
                                    data_line.append(logit_price['generateRate']['receiveLightRate']) # jzkh_l_unit_fee
#                                     print("3. jzkh_l_unit_fee=%d\n" % logit_price['generateRate']['receiveLightRate'])
                                else:
                                    data_line.append(logit_price['generateRate']['lightRate']) # jzkh_l_unit_fee
#                                     print("4. jzkh_l_unit_fee=%d\n" % logit_price['generateRate']['lightRate'])
                                # --------------------
                                if logit_price['generateRate'].get('receiveLightRate') is not None:
                                    data_line.append(logit_price['generateRate']['receiveLowestPrice']) # jzkh_low_fee
#                                     print("5. jzkh_low_fee=%d\n" % logit_price['generateRate']['receiveLightRate'])
                                else:
                                    data_line.append(logit_price['generateRate']['lowestPrice']) # jzkh_low_fee
#                                     print("6. jzkh_low_fee=%d\n" % logit_price['generateRate']['lowestPrice'])
                            else:
                                for _ in range(3):
                                    data_line.append('')
                            if logit_price.get('speed') is not None and logit_price['speed'].get('promiseQuickTime') is not None\
                                and logit_price['speed'].get('deliveryAddDays') is not None:
                                data_line.append(u'第%d天' % (logit_price['speed']['promiseQuickTime'] + 
                                                            logit_price['speed']['deliveryAddDays'])) # jzkh_day
#                                 print("7. jzkh_day=第%d天\n" % (logit_price['speed']['promiseQuickTime'] + 
#                                                             logit_price['speed']['deliveryAddDays']))
                            else:
#                                 print(logit_price['speed']['deliveryTime'], query_date)
                                diliver_date = logit_price['speed']['deliveryTime'].split('前')[0]
                                diliver_date = datetime.datetime.strptime(diliver_date, '%Y-%m-%d %H:%M')
                                data_line.append(u'第%d天' % ((diliver_date - query_date).days + 1)) # jzkh_day
#                                 print("8. jzkh_day=第%d天\n" % ((diliver_date - query_date).days + 1))
                            target_counter += 1
                        elif logit_price.__contains__('productName') and logit_price['productName'].find(u'精准汽运') >= 0:
                            if logit_price.get('generateRate') is not None:
                                if logit_price['generateRate'].get('receiveHeavyRate') is not None:
                                    data_line.append(logit_price['generateRate']['receiveHeavyRate']) # jzqy_h_unit_fee
#                                     print("9. jzqy_h_unit_fee=%.2f\n" % logit_price['generateRate']['receiveHeavyRate'])
                                else:
                                    data_line.append(logit_price['generateRate']['heavyRate']) # jzqy_h_unit_fee
#                                     print("10. jzqy_h_unit_fee=%.2f\n" % logit_price['generateRate']['heavyRate'])
                                # --------------------
                                if logit_price['generateRate'].get('receiveLightRate') is not None:
                                    data_line.append(logit_price['generateRate']['receiveLightRate']) # jzqy_l_unit_fee
#                                     print("11. jzqy_l_unit_fee=%d\n" % logit_price['generateRate']['receiveLightRate'])
                                else:
                                    data_line.append(logit_price['generateRate']['lightRate']) # jzqy_l_unit_fee
#                                     print("12. jzqy_l_unit_fee=%d\n" % logit_price['generateRate']['lightRate'])
                                # --------------------
                                if logit_price['generateRate'].get('receiveLowestPrice') is not None:
                                    data_line.append(logit_price['generateRate']['receiveLowestPrice']) # jzqy_low_fee
#                                     print("13. jzqy_low_fee=%d\n" % logit_price['generateRate']['receiveLowestPrice'])
                                else:
                                    data_line.append(logit_price['generateRate']['lowestPrice']) # jzqy_low_fee
#                                     print("14. jzqy_low_fee=%d\n" % (logit_price['generateRate']['lowestPrice']))
                            else:
                                for _ in range(3):
                                    data_line.append('')
                            if logit_price.get('speed') is not None :
                                quick_time = logit_price['speed'].get('promiseQuickTime')
                                slow_time = logit_price['speed'].get('promiseSlowTime')
                                quick_time = np.ceil(quick_time/24) if logit_price['speed'].get('unitOfPromiseQuickTime') == 'HOURS'\
                                                                    else quick_time
                                slow_time = np.ceil(slow_time/24) if logit_price['speed'].get('unitPromiseOfSlowTime') == 'HOURS' \
                                                                    else slow_time
#                                 print(quick_time, slow_time)
                                if quick_time is not None and slow_time is not None:
                                    if quick_time == slow_time:
                                        data_line.append(u'第%d天' % (quick_time)) # jzqy_day
#                                         print("15. jzqy_day=第%d天\n" % (quick_time))
                                    else:
                                        data_line.append(u'第%d天到第%d天' % (quick_time, slow_time)) # jzqy_day
#                                         print("16. jzqy_day=第%d天到第%d天\n" % (quick_time, slow_time))
                                else:
                                    diliver_date = logit_price['speed']['deliveryTime'].split('前')[0]
                                    diliver_date = datetime.datetime.strptime(diliver_date, '%Y-%m-%d')
                                    data_line.append(u'第%d天' % ((diliver_date - query_date).days + 1)) # jzqy_day
#                                     print(diliver_date, query_date)
#                                     print("17. jzqy_day=第%d天\n" % ((diliver_date - query_date).days + 1))
                            else:
                                data_line.append('')
                            target_counter += 1
                    if target_counter < 2:
                        for _ in range(4*(2-target_counter)):
                            data_line.append('')
                else:
                    for _ in range(8):
                        data_line.append('')
                if isinstance(data[i].get('expressPublishPrices'), list):
                    target_counter = 0
                    for express_price in data[i]['expressPublishPrices']:
                        if target_counter >= 2: break
                        if express_price.__contains__('productName') and express_price['productName'].find(u'3.60特惠件') >= 0:
                            if express_price.get('expressRate') is not None:
                                data_line.append(express_price['expressRate'].get('groundPrice')) # th_below_fee
                                data_line.append(express_price['expressRate'].get('rateOfStage1')) # th_unit_fee
                                data_line.append(express_price['expressRate'].get('groundPrice')) # th_low_fee
#                                 print("18. %d, %d, %d\n" % (express_price['expressRate'].get('groundPrice'),
#                                                               express_price['expressRate'].get('rateOfStage1'),
#                                                               express_price['expressRate'].get('groundPrice')))
                            else:
                                for _ in range(3):
                                    data_line.append('')
                            if express_price.get('speed') is not None:
                                if express_price.get('isNew') == 'new' and express_price['speed'].get('deliveryTime') is None:
                                    data_line.append(express_price['speed'].get('promiseArriveTime')) # th_day
#                                     print("19. th_day=%s\n" % (express_price['speed'].get('promiseArriveTime')))
                                elif express_price['speed'].get('deliveryTime') is not None:
                                    diliver_date = express_price['speed']['deliveryTime'].split('前')[0]
                                    diliver_date = datetime.datetime.strptime(diliver_date, '%Y-%m-%d')
                                    data_line.append(u'第%d天' % ((diliver_date - query_date).days + 1)) # th_day
#                                     print(diliver_date, query_date, (diliver_date - query_date).days + 1)
#                                     print("20. 第%d天\n" % ((diliver_date - query_date).days + 1))
                            else:
                                data_line.append('')
                            target_counter += 1
                        elif express_price.__contains__('productName') and express_price['productName'].find(u'3.60特重件') >= 0:
                            if express_price.get('expressRate') is not None:
                                data_line.append(express_price['expressRate'].get('groundPrice')) # tz_below_fee
                                data_line.append(express_price['expressRate'].get('rateOfStage1')) # tz_unit_fee
                                data_line.append(express_price['expressRate'].get('groundPrice')) # tz_low_fee
#                                 print("21. %d, %d, %d\n" % (express_price['expressRate'].get('groundPrice'),
#                                                               express_price['expressRate'].get('rateOfStage1'),
#                                                               express_price['expressRate'].get('groundPrice')))
                            else:
                                for _ in range(3):
                                    data_line.append('')
                            if express_price.get('speed') is not None:
                                if express_price.get('isNew') == 'new' and express_price['speed'].get('deliveryTime') is None:
                                    data_line.append(express_price['speed'].get('promiseArriveTime')) # tz_day
#                                     print("22. th_day=%s\n" % (express_price['speed'].get('promiseArriveTime')))
                                elif express_price['speed'].get('deliveryTime') is not None:
                                    diliver_date = express_price['speed']['deliveryTime'].split('前')[0]
                                    diliver_date = datetime.datetime.strptime(diliver_date, '%Y-%m-%d')
                                    data_line.append(u'第%d天' % ((diliver_date - query_date).days + 1)) # tz_day
#                                     print(diliver_date, query_date, (diliver_date - query_date).days + 1)
#                                     print("23. 第%d天\n" % ((diliver_date - query_date).days + 1))
                            else:
                                data_line.append('')
                            target_counter += 1
                    if target_counter < 2:
                        for _ in range(4*(2-target_counter)):
                            data_line.append('')
                else:
                    for _ in range(8):
                        data_line.append('')
                
                fp.write('%s\n'% '\t'.join([str(col) for col in data_line]))
        print('Done')
        

Processing 辽宁省/朝阳市.pkl...Done
Processing 辽宁省/沈阳市.pkl...Done
Processing 辽宁省/鞍山市.pkl...Done
Processing 辽宁省/锦州市.pkl...Done
Processing 辽宁省/盘锦市.pkl...Done
Processing 辽宁省/葫芦岛市.pkl...Done
Processing 辽宁省/本溪市.pkl...Done
Processing 辽宁省/丹东市.pkl...Done
Processing 辽宁省/抚顺市.pkl...Done
Processing 辽宁省/营口市.pkl...Done
Processing 辽宁省/铁岭市.pkl...Done
Processing 辽宁省/大连市.pkl...Done
Processing 辽宁省/辽阳市.pkl...Done
Processing 辽宁省/阜新市.pkl...Done
Processing 新疆维吾尔自治区/克拉玛依市.pkl...Done
Processing 新疆维吾尔自治区/阿拉尔市.pkl...Done
Processing 新疆维吾尔自治区/克孜勒苏柯尔克孜自治州.pkl...Done
Processing 新疆维吾尔自治区/喀什地区.pkl...Done
Processing 新疆维吾尔自治区/阿勒泰地区.pkl...Done
Processing 新疆维吾尔自治区/阿克苏地区.pkl...Done
Processing 新疆维吾尔自治区/吐鲁番市.pkl...Done
Processing 新疆维吾尔自治区/昌吉回族自治州.pkl...Done
Processing 新疆维吾尔自治区/巴音郭楞蒙古自治州.pkl...Done
Processing 新疆维吾尔自治区/石河子市.pkl...Done
Processing 新疆维吾尔自治区/乌鲁木齐市.pkl...Done
Processing 新疆维吾尔自治区/塔城地区.pkl...Done
Processing 新疆维吾尔自治区/博尔塔拉蒙古自治州.pkl...Done
Processing 新疆维吾尔自治区/和田地区.pkl...Done
Processing 新疆维吾尔自治区/五家渠市.pkl...Done
Processing 新疆维吾

Processing 江西省/九江市.pkl...Done
Processing 江西省/赣州市.pkl...Done
Processing 江西省/吉安市.pkl...Done
Processing 江西省/鹰潭市.pkl...Done
Processing 江西省/上饶市.pkl...Done
Processing 江西省/新余市.pkl...Done
Processing 四川省/自贡市.pkl...Done
Processing 四川省/成都市.pkl...Done
Processing 四川省/泸州市.pkl...Done
Processing 四川省/雅安市.pkl...Done
Processing 四川省/绵阳市.pkl...Done
Processing 四川省/宜宾市.pkl...Done
Processing 四川省/广元市.pkl...Done
Processing 四川省/资阳市.pkl...Done
Processing 四川省/广安市.pkl...Done
Processing 四川省/南充市.pkl...Done
Processing 四川省/凉山彝族自治州.pkl...Done
Processing 四川省/德阳市.pkl...Done
Processing 四川省/眉山市.pkl...Done
Processing 四川省/阿坝藏族羌族自治州.pkl...Done
Processing 四川省/内江市.pkl...Done
Processing 四川省/甘孜藏族自治州.pkl...Done
Processing 四川省/攀枝花市.pkl...Done
Processing 四川省/达州市.pkl...Done
Processing 四川省/巴中市.pkl...Done
Processing 四川省/乐山市.pkl...Done
Processing 四川省/遂宁市.pkl...Done
Processing 广东省/茂名市.pkl...Done
Processing 广东省/韶关市.pkl...Done
Processing 广东省/东莞市.pkl...Done
Processing 广东省/惠州市.pkl...Done
Processing 广东省/清远市.pkl...Done
Processing 广东省/河源市.pkl...

In [3]:
data[0]['fromAddr'].split('-')

['浙江省', '舟山市', '嵊泗县']

In [5]:
data_line

['河北省', '廊坊市', '固安县', '澳门特别行政区', '澳门', '澳门县', '第4天']

In [4]:
print(from_province)
print(from_city)
print(from_area)
print(to_province)
print(to_city)
print(to_area)         
print(jzkh_h_unit_fee)
print(jzkh_l_unit_fee)
print(jzkh_day)
print(jzkh_low_fee)
print(jzqy_h_unit_fee)
print(jzqy_l_unit_fee)
print(jzqy_day)
print(jzqy_low_fee)
print(th_below_fee)
print(th_unit_fee)
print(th_low_fee)
print(th_day)

NameError: name 'from_province' is not defined

In [31]:
df = pd.DataFrame({
    'from_province': from_province,
    'from_city': from_city,
    'from_area': from_area,
    'to_province': to_province,
    'to_city': to_city,
    'to_area': to_area,
    'jzkh_h_unit_fee': jzkh_h_unit_fee,
    'jzkh_l_unit_fee': jzkh_l_unit_fee,
    'jzkh_day': jzkh_day,
    'jzkh_low_fee': jzkh_low_fee,
    'jzqy_h_unit_fee': jzqy_h_unit_fee,
    'jzqy_l_unit_fee': jzqy_l_unit_fee,
    'jzqy_day': jzqy_day,
    'jzqy_low_fee': jzqy_low_fee,
    'th_below_fee': th_below_fee,
    'th_unit_fee': th_unit_fee,
    'th_low_fee': th_low_fee,
    'th_day': th_day
})

In [38]:
df.to_csv('test.csv', index=None)

In [39]:
df

,from_province,from_city,from_area,to_province,to_city,to_area,jzkh_h_unit_fee,jzkh_l_unit_fee,jzkh_day,jzkh_low_fee,jzqy_h_unit_fee,jzqy_l_unit_fee,jzqy_day,jzqy_low_fee,th_below_fee,th_unit_fee,th_low_fee,th_day
0,广东省,深圳市,罗湖区,浙江省,舟山市,嵊泗县,3.0,630,第5天,65,2.55,536,第6天,55,21,5,21,第4天
1,广东省,深圳市,罗湖区,浙江省,舟山市,岱山县,2.7,567,第3天,65,2.25,473,第4天,55,21,5,21,第2天
2,广东省,深圳市,罗湖区,浙江省,衢州市,开化县,2.7,567,第4天,65,2.25,473,第6天,55,21,5,21,第3天
3,广东省,深圳市,罗湖区,浙江省,衢州市,衢江区,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
4,广东省,深圳市,罗湖区,浙江省,衢州市,江山市,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
5,广东省,深圳市,罗湖区,浙江省,衢州市,柯城区,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
6,广东省,深圳市,罗湖区,浙江省,衢州市,常山县,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
7,广东省,深圳市,罗湖区,浙江省,衢州市,龙游县,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
8,广东省,深圳市,罗湖区,浙江省,金华市,义乌市,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
9,广东省,深圳市,罗湖区,浙江省,金华市,婺城区,2.6,546,第4天,60,2.15,452,第6天,50,21,5,21,第3天
